## TOC:
* [Datasets](#datasets-bullet)
* [Logging Settings](#logging-bullet)
* [Request](#requests-bullet)
* [Save Results](#results-bullet)

In [1]:
import json
import yaml
import requests
import getpass
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

## Datasets <a class="anchor" id="datasets-bullet"></a>

In [2]:
testing_datasets = [
    {
        "file_name": "https://github.com/ourownstory/neuralprophet-data/raw/main/datasets/air_passengers.csv",
        "format": "csv",
        "time_col_index": 0,
        "target_col_index": -1,
        "exovar": False,
    },
    {
        "file_name": "../datasets/air_quality/BeijingPM25_0.csv",
        "format": "csv",
        "time_col_index": 0,
        "target_col_index": -1,
        "exovar": False,
    }
]

In [3]:
def prepare_dataset(dataset_info):
    if dataset_info["format"] == "csv":
        train_df = pd.read_csv(dataset_info["file_name"])
        time_col = train_df.columns[dataset_info["time_col_index"]]
        target_col = train_df.columns[dataset_info["target_col_index"]]
        train_df[time_col] = train_df[time_col].astype(str)
        train_df[target_col] = train_df[target_col].astype(float)
        train_data = []
        for value in train_df.values:
            train_data.append(list(value))
    return train_data

## Logging Settings <a class="anchor" id="logging-bullet"></a>

In [4]:
LOG_RESULTS = True

In [5]:
user_name = getpass.getuser()

In [6]:
output_file_name = "./test_logs/test_logs_{}.csv".format(user_name)
print(output_file_name)

column_names = ["file_name", "model_request", "user", "time", "response", "type", "metrics"]
log_df = pd.DataFrame(columns=column_names)

./test_logs/test_logs_studio-lab-user.csv


## Request <a class="anchor" id="requests-bullet"></a>

Define Model Request (optional)

In [7]:
# Customized model request
model_request = {
    "type": "meta_lr",  # 'meta_wa'
    "scorers": ["smape", "mape"],
    "params": {
        "preprocessors": [
            {"type": "dartsimputer", "params": {"strategy": "mean"}},
            # {'type': 'simpleimputer', 'params': {'strategy': 'mean'}},
            {"type": "minmaxscaler"},
        ],
        "base_models": [
            {"type": "darts_naive"},
            {"type": "darts_seasonalnaive"},
            {"type": "darts_autotheta"},
            {"type": "darts_autoets"},
            {"type": "darts_autoarima"},
            #{"type": "darts_tbats"},
            #{"type": "darts_linearregression"},
            # {'type': 'darts_lightgbm'},
            # {'type': 'darts_rnn'},
        ],
    },
}

In [8]:
with open("url.yaml", "r") as file:
    url_dict = yaml.safe_load(file)

# URL to our SYBIL AWS service
protocol = url_dict["protocol"]
host = url_dict["host"]
port = url_dict["port"]
endpoint = "train"

url = "%s://%s:%s/%s" % (protocol, host, str(port), endpoint)

In [10]:
for dataset in testing_datasets:
    train_data = prepare_dataset(dataset)
    api_json = {
        "data": train_data,
        "model": model_request,  # (optional) can be commented out
    }
    start_time = time.time()
    response = requests.post(url, json=api_json)
    exc_time = time.time() - start_time
    model_req = api_json["model"] if "model" in api_json.keys() else "default"
    new_row = pd.DataFrame(
        [
            {
                "file_name": dataset["file_name"],
                "model_request": model_req,
                "user": user_name,
                "time": exc_time,
                "response": response.status_code,
                "type": response.json()['type'],
                "metrics": response.json()['metrics']
            }
        ]
    )
    log_df = pd.concat([log_df, new_row], ignore_index=True)
    print(response)

<Response [200]>
<Response [200]>


## Save Results <a class="anchor" id="results-bullet"></a>

In [11]:
# Logs
log_df

,file_name,model_request,user,time,response,type,metrics
0,https://github.com/ourownstory/neuralprophet-d...,"{'type': 'meta_lr', 'scorers': ['smape', 'mape...",studio-lab-user,57.078702,200,meta_lr,"[{'type': 'smape', 'value': 3.6943215019536004..."
1,../datasets/air_quality/BeijingPM25_0.csv,"{'type': 'meta_lr', 'scorers': ['smape', 'mape...",studio-lab-user,1111.253141,200,meta_lr,"[{'type': 'smape', 'value': 83.30772951856794}..."


In [ ]:
log_df['model_request'].iloc[0]

In [ ]:
# Broken runs
log_df[log_df['response'] != 200]

In [ ]:
if LOG_RESULTS:
    if os.path.exists(output_file_name):
        existing_log_df = pd.read_csv(output_file_name)
        updated_df = pd.concat([existing_log_df, log_df], ignore_index=True)
    else:
        updated_df = log_df
    updated_df.to_csv(output_file_name, index=False)